In [1]:
import pandas as pd
import openpyxl
from openai import OpenAI, OpenAIError
import re
import time

#Cargamos el archivo de Excel en un DataFrame
problemas = pd.read_excel('/content/drive/MyDrive/Traducción_DataSets.xlsx')
dataset = pd.DataFrame(problemas)

#Inicializamos el cliente de OpenAI (corregido)
client = OpenAI(api_key='')

#Prueba de conexión con OpenAI
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "¿Cuánto es 2+2?"}]
)
print("Prueba API:", response.choices[0].message.content)  # Acceso corregido

#Función para resolver los problemas aritméticos
def solver(cuerpo, pregunta, prompt):
    try:
        if pd.isna(cuerpo) or pd.isna(pregunta):
            return "El cuerpo o la pregunta está vacío."

        respuesta = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un asistente que resuelve problemas matemáticos."},
                {"role": "user", "content": f"{cuerpo} {pregunta} {prompt}"}
            ]
        )

        #Acceso al contenido de la respuesta
        return respuesta.choices[0].message.content

    except OpenAIError as e:
        return f"Error en la API de OpenAI: {str(e)}"
    except Exception as e:
        return f"Error desconocido: {str(e)}"

#Tomamos un subconjunto del Dataset
dataset_n = dataset.iloc[:,:]

answers = pd.DataFrame(dataset_n)

#Aplicamos la función solver a cada fila
answers['Solucion'] = answers.apply(lambda row: solver(row.get('Cuerpo', ''), row.get('Pregunta', ''), row.get('Prompt')), axis=1)

def numero(respuesta):

  sol = re.findall(r'-?\d+(?:\.\d+)?', respuesta)
  if sol:
    return sol[-1]
  else:
    return None

"""def numero(respuesta):
  sol = re.findall(r'\d+\.?\d*', respuesta)
  return float(sol[-1]) if sol else None"""

answers['sol_num'] = answers['Solucion'].apply(numero)

answers['Respuesta_GPT'] = answers['sol_num'].round(2)

#answers['Respuesta_GPT'] = answers['Respuesta_GPT'].astype(str)

#Configuramos para mostrar todo el contenido de las columnas
pd.set_option('display.max_colwidth', None)

#Mostramos el resultado
#print(answers[['Cuerpo', 'Pregunta', 'Respuesta_GPT', 'Respuesta']])

answers['Acierto'] = answers['Respuesta_GPT'].astype(float) == answers['Respuesta'].astype(float)

pd.set_option('display.max_rows', None)

# Mostramos el DataFrame con la nueva columna 'Acierto'
print(answers[['Respuesta_GPT','Respuesta','Acierto']])

correct = answers['Acierto'].sum()

print(f'Número de respuestas correctas: {correct}')

c = len(answers)

#print(f'Número de respuestas totales: {c}')

accuracy = correct / c*100

print(f'Porcentaje de aciertos: {accuracy:.2f}%')



Prueba API: 2 + 2 = 4
   Respuesta_GPT  Respuesta  Acierto
0           15.5      15.50     True
1            203     203.00     True
2              9       9.00     True
3              6       6.00     True
4             26      26.00     True
5             18      18.00     True
6           None      39.00    False
7              7       7.00     True
8             14      14.00     True
9             38      37.48    False
10          None       5.00    False
11          None       2.00    False
12            11      11.00     True
13          None       2.00    False
14             9       9.00     True
15          None       8.00    False
16            55      55.00     True
17            31      31.00     True
18            22      22.00     True
19            35      35.00     True
20            17      17.00     True
21            18      18.00     True
22             2       3.00    False
23            45      45.00     True
24             7       7.00     True
25           151